# Vertically Integrated Moisture Flux Convergence

This script illustrates calculating vertically integrated moisture flux convergence (MFC) for a dataset with a constant pressure level system.

To calculate this, we must first leverage [`geocat.comp.meteorology.delta_pressure`](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.meteorology.delta_pressure.html) for the vertical integration, and then [`geocat.comp.gradient.gradient`](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.gradient.gradient.html#geocat.comp.gradient.gradient) to get our gradients on the surface of the WGS84 ellipsoid.

In [ ]:
import xarray as xr
import numpy as np
import geocat.comp as gc
import geocat.datafiles as gcd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Load Data

Here we load in a dummy Xarray Dataset with 2d zonal and meridional wind componenets (`u_wind` and `v_wind`), 3d water vapor (`q_vapor`), a 2d surface pressure array (`surface_pressure`), and a 1d pressure level system (`pressure_lev`).

In [ ]:
ds = xr.open_dataset(gcd.get("netcdf_files/dummy_uvqp_data.nc"))
ds

#  Vertical Integration

To do a mass-weighted vertical integration of our 3d water vapor, we need to first get a layer mass weighting. This is our delta pressure levels, divide by the gravitational constant. Then, multiply the layer mass weighting by the water vapor variable, and integrate vertially. 

In [ ]:
delta_pressure_levels = gc.meteorology.delta_pressure(pressure_lev=ds.pressure_lev, surface_pressure=ds.surface_pressure)

g = 9.80665 # gravitational acceleration (m s-2)
layer_mass_weighting = delta_pressure_levels / g # Layer Mass Weighting
layer_mass_weighting .attrs["units"] = "kg m-2"

mass_weighted_vapor = ds.q_vapor * layer_mass_weighting # mass weighted 'q'
iq = mass_weighted_vapor.sum(dim="lev") # Vertically Integrated Vapor
iq.attrs["units"] = "g m-2"

# Calculate Moisture Flux Convergence (MFC)

The formula for MFC is as follows:

$MFC =  -u*\frac{dq}{dx} - v*\frac{dq}{dy} - q(\frac{du}{dx} + \frac{dv}{dy})$

where u represents the zonal wind component, v is the meridional wind componenet, and q is the moisture content (in our case, vertically integrated).

The first two terms of this equation, $-u*\frac{dq}{dx} - v*\frac{dq}{dy}$, are together the advection term, and the last term, $q(\frac{du}{dx} + \frac{dv}{dy})$, is our convergence term, with moisture flux convergence being the advection minus the convergence.

[Source: Banacos and Schultz, "Moisture Flux Convergence", NOAA 2004](https://www.spc.noaa.gov/publications/banacos/mfc-sls.pdf)

To do this calculation, we first need the gradients of our wind components and moisture. We are using Geocat-comp's [`gradient`](https://geocat-comp.readthedocs.io/en/latest/user_api/generated/geocat.comp.gradient.gradient.html#geocat.comp.gradient.gradient) method because of its improved accuracy for geospatial data over an Earth-like ellipsoid.

In [ ]:
du_dx, du_dy = gc.gradient(ds.u_wind) # (s-1)
dv_dx, dv_dy = gc.gradient(ds.v_wind)
dq_dx, dq_dy = gc.gradient(iq) # (g m-3)

In [ ]:
advection = (-u_wind * dq_dx) - (v_wind * dq_dy) # (g s-1 m-2)
convergence = iq * np.add(du_dx, dv_dy) # (g s-1 m-2)

mfc = advection - convergence # moisture flux convergence (g m-2 s-1)

## Plot Data

And finally we can view our moisture flux convergence.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(lon, lat, mfc, transform=ccrs.PlateCarree())
plt.title('Vertically Integrated Moisture Flux Convergence')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

cbar = plt.colorbar()
cbar.set_label('Moisture Flux Convergence (g m-2 s-1)')

ax.coastlines()


gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False;